In [4]:
import numpy as np
import os
import re
import ast
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

# === 1. CONFIGURATION ===
base_path = "data/"
inputs_txt = "week2/inputs.txt"
outputs_txt = "week2/outputs.txt"

# === 2. LOAD INPUTS ===
with open(inputs_txt, "r") as f:
    text = f.read().strip()

# Pisahkan jadi 2 batch: minggu 1 dan minggu 2
batches = re.findall(r'\[array\([^\]]+\)\]', text, re.DOTALL)
chunks = text.split("]\n[")  # pisah antar baris
latest_batch = chunks[-1]    # ambil batch terakhir (week 2)
text_clean = re.sub(r'array\(', '', latest_batch).replace(')', '')
new_inputs = ast.literal_eval("[" + text_clean.strip().strip('[]') + "]")
new_inputs = [np.array(x, dtype=float) for x in new_inputs]

# === 3. LOAD OUTPUTS ===
with open(outputs_txt, "r") as f:
    text_out = f.read().strip()
chunks_out = text_out.split("]\n[")
latest_batch_out = chunks_out[-1]
text_clean_out = latest_batch_out.replace("np.float64(", "").replace(")", "")
new_outputs = np.array(ast.literal_eval("[" + text_clean_out.strip().strip('[]') + "]"), dtype=float)

print(f"✅ Loaded {len(new_inputs)} new inputs and {len(new_outputs)} outputs (latest batch).")

# === 4. PROCESS EACH FUNCTION WITH SVM ===
for i in range(1, 9):
    folder = os.path.join(base_path, f"function_{i}")
    input_file = os.path.join(folder, "week2_inputs.npy")
    output_file = os.path.join(folder, "week2_outputs.npy")

    # Load data minggu lalu (hasil Linear Regression)
    X_prev = np.load(input_file)
    y_prev = np.load(output_file)

    # Gabungkan dengan hasil minggu lalu
    X_combined = np.vstack([X_prev, new_inputs[i - 1].reshape(1, -1)])  # 12 titik
    y_combined = np.append(y_prev, new_outputs[i - 1])

    # Simpan versi updated
    np.save(os.path.join(folder, "week3_inputs.npy"), X_combined)
    np.save(os.path.join(folder, "week3_outputs.npy"), y_combined)

    dim = X_combined.shape[1]
    print(f"\n=== Function {i} ({dim}D) ===")
    print(f"  Data size: {X_combined.shape}, Output range: [{y_combined.min():.4f}, {y_combined.max():.4f}]")

    # === TRAIN SVM MODEL ===
    model = make_pipeline(StandardScaler(), SVR(kernel='rbf', C=5, gamma='scale'))
    model.fit(X_combined, y_combined)

    # === GENERATE NEW QUERY ===
    candidates = np.random.uniform(0, 1, (2000, dim))
    preds = model.predict(candidates)
    best_idx = np.argmax(preds)
    best_query = np.clip(candidates[best_idx], 0.0, 1.0)

    # === OUTPUT RESULT ===
    query_str = "-".join([f"{x:.6f}" for x in best_query])
    print(f"  Best predicted output: {preds[best_idx]:.4f}")
    print(f"  Query to submit: {query_str}")

✅ Loaded 8 new inputs and 8 outputs (latest batch).

=== Function 1 (2D) ===
  Data size: (12, 2), Output range: [-0.0036, 0.0000]
  Best predicted output: -0.0018
  Query to submit: 0.167129-0.724522

=== Function 2 (2D) ===
  Data size: (12, 2), Output range: [-0.0656, 0.6112]
  Best predicted output: 0.7560
  Query to submit: 0.868661-0.995937

=== Function 3 (3D) ===
  Data size: (17, 3), Output range: [-0.3989, -0.0348]
  Best predicted output: -0.1209
  Query to submit: 0.387719-0.760476-0.452020

=== Function 4 (4D) ===
  Data size: (32, 4), Output range: [-32.6257, -4.0255]
  Best predicted output: -5.0023
  Query to submit: 0.348768-0.491521-0.414453-0.271505

=== Function 5 (4D) ===
  Data size: (22, 4), Output range: [0.1129, 1787.3908]
  Best predicted output: 85.2658
  Query to submit: 0.152397-0.865019-0.893083-0.890029

=== Function 6 (5D) ===
  Data size: (22, 5), Output range: [-2.5712, -0.7143]
  Best predicted output: -0.6079
  Query to submit: 0.566002-0.279903-0.52